# Algorithm 8: Confidence v2 (Boltz-2)

Enhanced confidence prediction for Boltz-2.

## Source Code Location
- **File**: `Boltz-Ref-src/boltz-official/src/boltz/model/modules/confidencev2.py`

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

def softmax(x, axis=-1):
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

In [ ]:
def plddt_head_v2(s, num_bins=50):
    """Enhanced pLDDT prediction."""
    N, c_s = s.shape
    
    # Additional MLP layers in v2
    s_norm = layer_norm(s)
    
    W1 = np.random.randn(c_s, c_s) * (c_s ** -0.5)
    h = np.maximum(0, s_norm @ W1)  # ReLU
    h = layer_norm(h)
    
    W2 = np.random.randn(c_s, num_bins) * (c_s ** -0.5)
    logits = h @ W2
    
    probs = softmax(logits)
    bin_centers = np.linspace(0, 100, num_bins)
    plddt = np.sum(probs * bin_centers, axis=-1)
    
    return logits, plddt

In [ ]:
def pae_head_v2(z, num_bins=64, max_error=32.0):
    """Enhanced PAE prediction."""
    c_z = z.shape[-1]
    
    z_norm = layer_norm(z)
    
    # Additional processing in v2
    W1 = np.random.randn(c_z, c_z) * (c_z ** -0.5)
    h = np.maximum(0, np.einsum('ijc,cd->ijd', z_norm, W1))
    h = layer_norm(h)
    
    W2 = np.random.randn(c_z, num_bins) * (c_z ** -0.5)
    logits = np.einsum('ijc,cb->ijb', h, W2)
    
    probs = softmax(logits)
    bin_centers = np.linspace(0, max_error, num_bins)
    pae = np.sum(probs * bin_centers, axis=-1)
    
    return logits, pae

In [ ]:
def confidence_module_v2(s, z, x_pred):
    """
    Confidence Module v2.
    
    Enhanced confidence prediction with structure awareness.
    
    Args:
        s: Single representation [N, c_s]
        z: Pair representation [N, N, c_z]
        x_pred: Predicted coordinates [N, 3]
    
    Returns:
        Dictionary of confidence scores
    """
    print(f"Confidence Module v2 (Boltz-2)")
    print(f"="*50)
    
    # Compute predicted distances for additional context
    diff = x_pred[:, None, :] - x_pred[None, :, :]
    pred_dist = np.sqrt(np.sum(diff ** 2, axis=-1) + 1e-8)
    
    # Embed distances
    dist_emb = pred_dist[:, :, np.newaxis] / 10.0  # Normalize
    
    # Augment pair representation
    W_dist = np.random.randn(1, z.shape[-1]) * 0.1
    z_aug = z + dist_emb @ W_dist
    
    # pLDDT
    plddt_logits, plddt = plddt_head_v2(s)
    print(f"pLDDT: mean={plddt.mean():.1f}, range=[{plddt.min():.1f}, {plddt.max():.1f}]")
    
    # PAE
    pae_logits, pae = pae_head_v2(z_aug)
    print(f"PAE: mean={pae.mean():.2f}A")
    
    # Compute pTM
    d0 = 1.24 * (len(s) - 15) ** (1/3) - 1.8
    d0 = max(d0, 0.5)
    tm_scores = 1 / (1 + (pae / d0) ** 2)
    ptm = tm_scores.max(axis=1).mean()
    print(f"pTM: {ptm:.3f}")
    
    return {
        'plddt': plddt,
        'pae': pae,
        'ptm': ptm,
    }

In [ ]:
# Test
print("Test: Confidence Module v2")
print("="*60)

N = 32
c_s = 128
c_z = 64

s = np.random.randn(N, c_s)
z = np.random.randn(N, N, c_z)
x_pred = np.random.randn(N, 3) * 10

confidence = confidence_module_v2(s, z, x_pred)

## Key Insights

1. **Structure-Aware**: Uses predicted distances
2. **Deeper Networks**: Additional MLP layers
3. **pTM Computation**: Proper TM-score based metric
4. **Better Calibration**: Improved uncertainty estimates